<a href="https://colab.research.google.com/github/David9857/VQA/blob/main/KAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
import spacy.cli
spacy.cli.download("en_vectors_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_vectors_web_lg')


In [ ]:
!mkdir ./results

In [ ]:
!python /content/drive/MyDrive/KAN/train.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/KAN/train.py", line 2, in <module>
    from core.exec import Execution
  File "/content/drive/.shortcut-targets-by-id/1qq5LCjZ3AGmL2Kv20y4s0nEVp9K7G-Mn/KAN/core/exec.py", line 2, in <module>
    from core.model.net import Net
ModuleNotFoundError: No module named 'core.model.net'


In [ ]:
!mkdir ./data
!cp /content/drive/MyDrive/data_path.zip ./data/
!cd ./data && unzip data_path.zip

In [ ]:
!cd ./data && unzip data_path.zip

In [ ]:
# !ls /content/data/data/ques_embeddings/bluebert
!cp -r /content/data/data/ques_embeddings/w2v /content/drive/MyDrive/VQA_path/ques_embeddings

In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy

/content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy


In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo | wc -l

32761


In [ ]:
!ls /content/data/data/ques_embeddings/bioelmo/7563.npy

/content/data/data/ques_embeddings/bioelmo/7563.npy


In [ ]:
!unzip -uq '/content/drive/MyDrive/data_path.zip' -d '/content/drive/MyDrive/data_path'

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import pandas as pd
import time
import pathlib
from utils.load_data import DataLoader
from utils.evaluation import AnswerEvaluator
from utils.training_toolkit import CustomSchedule, loss_function
from models.Transformer.transformers import VQATransformer
from models.Transformer.masks import create_masks

In [ ]:
class Image_Question_Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding,
                 rate=0.1 ,pretrained_cnn_type='inception', multiplier = 10):
        super(Image_Question_Encoder, self).__init__()
        self.maximum_position_encoding = maximum_position_encoding
        self.pretrained_CNN = pretrained_cnn(pretrained_cnn_type)
        self.fc1 = Dense(maximum_position_encoding * multiplier) # fully connected layer
        self.image_dense = Dense(d_model, activation='relu')
        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=maximum_position_encoding)
        self.concatenator = Concatenate(axis=1)
        self.conv = tf.keras.layers.Conv2D(1024, 3, padding='same', activation='relu')
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True))

    def call(self, question, img, training, mask):
        img_feature = self.pretrained_CNN(img)  # (7, 7, 1024)
        img_feature = self.conv(img_feature)
        img_shape = img_feature.shape[-1]
        img_feature = tf.keras.layers.Reshape((-1, img_shape))(img_feature)  # (49, 1024)
        img_feature = self.fc1(img_feature)  # (49, pe_input)
        img_feature = tf.transpose(img_feature, perm=(0, 2, 1))  # (Res, 49)
        img_feature = tf.keras.layers.Reshape((self.maximum_position_encoding, -1))(img_feature)
        img_feature = self.image_dense(img_feature)  # (pe_input, 512)
        question = self.bilstm(question)
        context = self.question_encoder(question, img_feature, training, mask)  # (batch, question_len, 512)
        # context = self.concatenator([img_feature, question_feature]) # （batch, 64 + question_len, 512)
        return context  # (64, 108, 512)

In [ ]:
transformer = VQATransformer(num_layers, d_model, num_heads, dff, vocab_size, pe_input, pe_output,
                          pretrained_cnn_type=cnn_type)

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, check_point_path, max_to_keep=5)

# Knowledge

In [2]:
pip install senticnet

     |████████████████████████████████| 51.9MB 142kB/s 


In [3]:
from senticnet.senticnet import SenticNet

sn = SenticNet()
concept_info = sn.concept('love')
polarity_label = sn.polarity_label('love')
polarity_value = sn.polarity_value('love')
moodtags = sn.moodtags('love')
semantics = sn.semantics('love')
sentics = sn.sentics('love')

In [6]:
print(concept_info)
print(polarity_label)
print(polarity_value)
print(moodtags)
print(semantics)
print(sentics)

{'polarity_label': 'positive', 'polarity_value': '0.83', 'sentics': {'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}, 'moodtags': ['#joy', '#eagerness'], 'semantics': ['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']}
positive
0.83
['#joy', '#eagerness']
['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']
{'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}
